In [2]:
import cv2
import numpy as np
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

In [21]:
video_cap = cv2.VideoCapture('Serso.mp4')

print(f' Video Captured: {video_cap.isOpened()}')
print(f' FPS: {video_cap.get(5)}')
print(f' Total Frames: {video_cap.get(7)}')

 Video Captured: True
 FPS: 25.0
 Total Frames: 5090.0


In [22]:
# Model initialization
base_options = python.BaseOptions(model_asset_path='blaze_face_short_range.tflite')
options = vision.FaceDetectorOptions(base_options=base_options)

detector = vision.FaceDetector.create_from_options(options)

In [23]:
# Detection & Bounding box visualization
def detect(model, frame, blur):
    rgb_frame = mp.Image(image_format = mp.ImageFormat.SRGB, data = frame)
    detection_result = model.detect(rgb_frame)
    processed_frame = frame.copy()
    
    for detected in detection_result.detections:
        bounds = detected.bounding_box
        left_upper = (bounds.origin_x, bounds.origin_y)
        right_bottom = (bounds.origin_x + bounds.width, bounds.origin_y + bounds.height)
        
        
        if (blur == True):
            crop_fragment = frame[bounds.origin_y : bounds.origin_y + bounds.height, 
                                  bounds.origin_x : bounds.origin_x + bounds.width]
            
            blurred_fragment = cv2.blur(crop_fragment, (65,65))
            
            processed_frame[bounds.origin_y : bounds.origin_y + bounds.height, 
                            bounds.origin_x : bounds.origin_x + bounds.width] = blurred_fragment
        else:
            # Bounding box
            object_category = detected.categories[0]
            score = round(object_category.score, 3)
            cv2.rectangle(processed_frame, left_upper, right_bottom, (255, 0, 255), 1)
            score_text_location = (10 + bounds.origin_x, -10 + bounds.origin_y)
            cv2.putText(processed_frame, f'score: {score}', score_text_location, cv2.FONT_HERSHEY_PLAIN, 1, (255, 0, 255))
        
    return processed_frame

def finalize_capture(vid_cap):
    vid_cap.release()
    cv2.destroyAllWindows()

In [24]:
# Detection loop
frame_num = 0

while (video_cap.isOpened()):
    frame_num += 1
    flag, frame = video_cap.read()
    
    if (flag == True):
        processed_frame = detect(detector, frame, blur = True)
        cv2.imshow(f'Frame', processed_frame)
        
        key = cv2.waitKey(40)
    
        if (key == ord('q')) or key == 27:
            break
    else:
        break

finalize_capture(video_cap)